<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto.

Se utilizará el **libro Crimen y Castigo de Fiódor Dostoievski** para generar los embeddings, es decir, que los vectores tendrán la forma en función de como ese libro haya sido escrito.

## Tarea: Desafío 2

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_fwf('/content/drive/MyDrive/Colab Notebooks/CEIA/04 - NLP/crimen_castigo.txt')
df.head()

,The Project Gutenberg eBook of El crimen y el castigo
0,This ebook is for the use of anyone anywhere i...
1,most other parts of the world at no cost and w...
2,"whatsoever. You may copy it, give it away or r..."
3,of the Project Gutenberg License included with...
4,at www.gutenberg.org. If you are not located i...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 17441


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and'],
 ['most',
  'other',
  'parts',
  'of',
  'the',
  'world',
  'at',
  'no',
  'cost',
  'and',
  'with',
  'almost',
  'no',
  'restrictions']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 17441


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 3491


### 3 - Entrenar embeddings

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50, # --> elegimnos 50 para mejorar el loss de solo 20
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1366022.625
Loss after epoch 1: 927166.125
Loss after epoch 2: 813926.75
Loss after epoch 3: 800569.25
Loss after epoch 4: 758213.75
Loss after epoch 5: 733440.0
Loss after epoch 6: 724095.0
Loss after epoch 7: 713694.5
Loss after epoch 8: 704878.5
Loss after epoch 9: 697171.5
Loss after epoch 10: 666027.0
Loss after epoch 11: 653967.0
Loss after epoch 12: 646339.0
Loss after epoch 13: 638995.0
Loss after epoch 14: 629980.0
Loss after epoch 15: 624184.0
Loss after epoch 16: 617456.0
Loss after epoch 17: 611838.0
Loss after epoch 18: 604402.0
Loss after epoch 19: 601051.0
Loss after epoch 20: 596127.0
Loss after epoch 21: 590732.0
Loss after epoch 22: 584452.0
Loss after epoch 23: 574724.0
Loss after epoch 24: 539224.0
Loss after epoch 25: 535178.0
Loss after epoch 26: 531408.0
Loss after epoch 27: 528872.0
Loss after epoch 28: 520470.0
Loss after epoch 29: 518552.0
Loss after epoch 30: 513606.0
Loss after epoch 31: 512116.0
Loss after epoch 32: 509292.0
Loss after e

(5664165, 9066300)

### 4 - Ensayar

#### Personajes
- Vamos a entender a algunos de los personajes.
- Tomamos el detalle de cada uno desde https://es.wikipedia.org/wiki/Crimen_y_castigo#Personajes y vemos si hay algo de relación usando estos embeddings.

In [29]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["rodia"], topn=10)

[('¡rodia', 0.4681476056575775),
 ('advierto', 0.4092000424861908),
 ('bendición', 0.3880685865879059),
 ('entiendo', 0.3645994961261749),
 ('querido', 0.36353421211242676),
 ('acercándose', 0.3627459406852722),
 ('dunetshka', 0.36102303862571716),
 ('dices', 0.35543087124824524),
 ('¡me', 0.3484506607055664),
 ('pulkeria', 0.347832590341568)]

- Vemos que sí se identifica con la hermana Dunia (Dunetshka).

In [31]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(positive=["sonia"], topn=10)

[('sofía', 0.4425055980682373),
 ('semenovna', 0.4233628511428833),
 ('atrevió', 0.39242061972618103),
 ('inmóvil', 0.3625342547893524),
 ('hacerla', 0.3550349771976471),
 ('vacilante', 0.348389208316803),
 ('cochero', 0.34706634283065796),
 ('marta', 0.34684106707572937),
 ('ronca', 0.3467690348625183),
 ('frunciendo', 0.34326937794685364)]

- También podemos decir que tiene que ver con su nombre Sofía, siendo Sonia su apodo.

In [40]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dunia"], topn=10)

[('acercándose', 0.3427722156047821),
 ('llamó', 0.34046465158462524),
 ('haría', 0.3387398421764374),
 ('levantar', 0.3341848850250244),
 ('cortés', 0.3323602080345154),
 ('temblorosa', 0.32303375005722046),
 ('suavemente', 0.3203120827674866),
 ('vendré', 0.3185602128505707),
 ('sospecha', 0.31687185168266296),
 ('vacilante', 0.3162606954574585)]

- Podemos decir que una de las cosas que es este personaje es ser cortés y de principios. Que haría muchas cosas por la familia.

#### Primera Parte
- La Primera parte habla de los conflictos que tiene Rodia con la anciana, entonces podemos buscar si tiene que ver la similitud con esto.

In [59]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["asesinar"], topn=10)

[('mató', 0.641558051109314),
 ('hachazos', 0.6349546909332275),
 ('oración', 0.5482273101806641),
 ('anciana', 0.5479016304016113),
 ('matar', 0.543271005153656),
 ('denunciarse', 0.5406516790390015),
 ('huir', 0.5073142051696777),
 ('«porque', 0.5044224858283997),
 ('nuevos', 0.5042123794555664),
 ('ayudar', 0.5026456713676453)]

- Vemos que el asesinato tiene que ver con la anciana.

#### Segunda Parte
- Enfermedad o delirio del personaje principal.

In [60]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["enfermedad"], topn=10)

[('hipocondría', 0.47173386812210083),
 ('deberes', 0.4527125656604767),
 ('ánimo', 0.44444015622138977),
 ('coronel', 0.4418908953666687),
 ('vater', 0.43170273303985596),
 ('casos', 0.4189532697200775),
 ('cliente', 0.4108167290687561),
 ('proceder', 0.40881964564323425),
 ('juventud', 0.40820151567459106),
 ('psicológico', 0.40748754143714905)]

- Tiene que ver con la hipocondría que sufre el personaje.

#### Tercera Parte
- Tiene que ver con la entrevista del personaje principal.

In [61]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["entrevista"], topn=10)

[('preocupación', 0.454860121011734),
 ('ternura', 0.45182764530181885),
 ('hipótesis', 0.44893598556518555),
 ('empresa', 0.44771280884742737),
 ('ilusión', 0.4386240243911743),
 ('tranquilidad', 0.43831413984298706),
 ('contaba', 0.43759408593177795),
 ('impaciencia', 0.434815913438797),
 ('pasada', 0.42722970247268677),
 ('bendición', 0.42561882734298706)]

- Vemos que cuenta bastante bien como el personaje se siente.

#### Título del libro
- Podemos ver con qué se puede relacionar mejor las palabras del título del libro.

In [41]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["crimen"], topn=10)

[('castigo', 0.44349679350852966),
 ('bajeza', 0.4409492313861847),
 ('ebook', 0.4216935634613037),
 ('literary', 0.41405874490737915),
 ('ambiente', 0.40576669573783875),
 ('inconveniente', 0.403408944606781),
 ('autor', 0.3969263732433319),
 ('cometer', 0.39664629101753235),
 ('conseguido', 0.3941151201725006),
 ('material', 0.39339733123779297)]

In [42]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["castigo"], topn=10)

[('ebook', 0.5908800959587097),
 ('móvil', 0.5164549946784973),
 ('by', 0.5094512104988098),
 ('¡las', 0.5034210085868835),
 ('suficiente', 0.4962206780910492),
 ('full', 0.4879506826400757),
 ('produced', 0.4878753423690796),
 ('license', 0.48192059993743896),
 ('using', 0.47848770022392273),
 ('an', 0.4773859977722168)]

- Vemos en este caso, que trasciende el texto del libro como tal.
- Esto es así ya que al bajar el libro, tiene mucho contenido respecto a la descripción del libro, bajado del proyecto Gutenberg https://www.gutenberg.org/ebooks/61851, más que a lo que dice.

In [51]:
# Revisamos las primeras palabras del texto y vemos que son las que describen el libro
df.head(50)

,The Project Gutenberg eBook of El crimen y el castigo
0,This ebook is for the use of anyone anywhere i...
1,most other parts of the world at no cost and w...
2,"whatsoever. You may copy it, give it away or r..."
3,of the Project Gutenberg License included with...
4,at www.gutenberg.org. If you are not located i...
5,you will have to check the laws of the country...
6,before using this eBook.
7,Title: El crimen y el castigo
8,Author: Fyodor Dostoyevsky
9,Translator: Pedro Pedraza y Páez


- Recién la primera parte del libro empieza en el row 42, lo cual hace ver que hubiese sido bueno cortar esto del análisis.

In [43]:
# Podemos comparar los vectores numéricos con la búsqueda de palabra per se
# el método `get_vector` permite obtener los vectores:
vector_rodia = w2v_model.wv.get_vector("rodia")
print(vector_rodia)

[-4.53214124e-02  2.82939583e-01 -1.55929267e-01 -4.02784228e-01
 -2.64932871e-01 -4.54302095e-02  3.03710580e-01  4.31576520e-01
  2.59137541e-01 -9.99177322e-02  1.86456993e-01 -5.17239273e-01
 -1.12113014e-01 -6.35536373e-01 -2.15075817e-02 -1.51097149e-01
  4.27151233e-01  2.69284576e-01  4.44830745e-01 -5.60064077e-01
 -7.13104308e-01 -2.09892750e-01  1.59694105e-01  1.61846519e-01
 -1.70965806e-01  2.15081796e-01  2.31238127e-01 -3.94093767e-02
 -1.74979582e-01  2.19926193e-01  2.07011893e-01  2.58682579e-01
 -4.60161641e-02  4.43608046e-01 -1.49647802e-01 -3.09071392e-01
  2.62633830e-01 -6.55837059e-01  1.41712546e-01  5.39934397e-01
 -5.44731431e-02  1.44865260e-01 -5.95771484e-02 -6.61179662e-01
  1.52387530e-01 -1.04402363e-01  9.12990630e-01  5.47845423e-01
  3.61721247e-01 -2.94709623e-01  2.97008067e-01 -2.76270270e-01
 -7.50170723e-02  6.50844425e-02 -6.13548517e-01 -1.79101955e-02
 -4.22723591e-01 -2.15398923e-01 -4.93738838e-02  2.16727987e-01
 -7.53361434e-02 -3.47483

In [44]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_rodia)

[('rodia', 1.0),
 ('¡rodia', 0.4681476056575775),
 ('advierto', 0.4092000424861908),
 ('bendición', 0.38806861639022827),
 ('entiendo', 0.3645994961261749),
 ('querido', 0.36353421211242676),
 ('acercándose', 0.3627459406852722),
 ('dunetshka', 0.36102303862571716),
 ('dices', 0.35543087124824524),
 ('¡me', 0.3484507203102112)]

In [45]:
# Comparando con lo visto arriba:
w2v_model.wv.most_similar(positive=["rodia"], topn=10)

[('¡rodia', 0.4681476056575775),
 ('advierto', 0.4092000424861908),
 ('bendición', 0.3880685865879059),
 ('entiendo', 0.3645994961261749),
 ('querido', 0.36353421211242676),
 ('acercándose', 0.3627459406852722),
 ('dunetshka', 0.36102303862571716),
 ('dices', 0.35543087124824524),
 ('¡me', 0.3484506607055664),
 ('pulkeria', 0.347832590341568)]

### 5 - Visualizar agrupación de vectores

In [46]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [47]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [48]:
# Aislamos nombre y apellido de uno de los personajes
MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

- Haciendo zoom en parte del texto, vemos que Pullería y Alexandrovna están relativamente cerca, dejando en claro que son el nombre y el apellido de uno de los personajes.

In [49]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [50]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### 6 - Conclusiones

- Podemos decir que usando este método Word2Vec, los embeddings de los personajes tienen relación semántica con las palabras similares.
- Al poder graficarlos, vemos que reduciendo dimensionalidad a 2D, tenemos la posibilidad de ubicar tanto nombre y apellido de un personaje como puntos cercanos del plano.
- Podemos agregar que buscando tan solo 1 palabra en cada una de las partes del libros, se puede apreciar la similitud en varias palabras con el contexto de lo que realmente pasa en el libro. Así, la similitud semántica es aparente.
- Al buscar el título del libro dentro del texto en cuestión, vemos que también hay mucha similitud con las palabras que son ajenas al text. Estas palabras forman parte del detalle de propaganda y descripción del Proyecto Gutenberg. De esta forma, se abre una opotunidad para chequear y "filtrar" este tipo de palabras del corpus.